In [ ]:
# Install required libraries
!pip install gradio pandas numpy folium

# Import libraries
import gradio as gr
import pandas as pd
import numpy as np
import folium
from branca.element import Figure

# Function to generate beach data
def generate_beach_data():
    data = {
        "Beach": ["Beach A", "Beach B", "Beach C", "Beach D"],
        "Temperature (°C)": np.random.randint(25, 35, 4),
        "Wind Speed (km/h)": np.random.randint(5, 20, 4),
        "Crowd Level": np.random.choice(["Low", "Medium", "High"], 4),
        "Water Quality": np.random.choice(["Good", "Moderate", "Poor"], 4),
        "Latitude": [28.6139, 28.7041, 12.9716, 19.0760],  # Example latitudes
        "Longitude": [77.2090, 77.1025, 77.5946, 72.8777],  # Example longitudes
    }
    return pd.DataFrame(data)

# Function to display beach data
def display_beach_data():
    df = generate_beach_data()
    return df

# Function to display an interactive map
def display_map():
    df = generate_beach_data()
    # Create a map
    fig = Figure(width=550, height=350)
    map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)
    fig.add_child(map)

    # Add markers for each beach
    for _, row in df.iterrows():
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=(
                f"<b>Beach:</b> {row['Beach']}<br>"
                f"<b>Temperature:</b> {row['Temperature (°C)']} °C<br>"
                f"<b>Wind Speed:</b> {row['Wind Speed (km/h)']} km/h<br>"
                f"<b>Crowd Level:</b> {row['Crowd Level']}<br>"
                f"<b>Water Quality:</b> {row['Water Quality']}"
            ),
            icon=folium.Icon(color="blue" if row["Water Quality"] == "Good" else "red"),
        ).add_to(map)

    return map._repr_html_()

# Function to predict crowd level based on user inputs
def predict_crowd(temp, wind):
    if temp > 30 and wind < 10:
        return "High"
    elif temp < 30 and wind > 15:
        return "Low"
    else:
        return "Medium"

# Create Gradio interface
with gr.Blocks() as app:
    gr.Markdown("## Enhancing Beach Tourism Experiences")

    # Display real-time beach data
    gr.Markdown("### Current Beach Conditions")
    data_output = gr.DataFrame(
        headers=["Beach", "Temperature (°C)", "Wind Speed (km/h)", "Crowd Level", "Water Quality"],
        label="Beach Data",
    )
    data_button = gr.Button("Generate Beach Data")
    data_button.click(display_beach_data, outputs=data_output)

    # Display interactive map
    gr.Markdown("### Interactive Map of Beaches")
    map_output = gr.HTML(label="Interactive Map")
    map_button = gr.Button("Show Map")
    map_button.click(display_map, outputs=map_output)

    # Crowd prediction feature
    gr.Markdown("### Predict Crowd Level")
    temp_input = gr.Slider(20, 40, step=1, label="Select Temperature (°C)")
    wind_input = gr.Slider(0, 30, step=1, label="Select Wind Speed (km/h)")
    predict_button = gr.Button("Predict")
    crowd_output = gr.Textbox(label="Predicted Crowd Level")
    predict_button.click(predict_crowd, inputs=[temp_input, wind_input], outputs=crowd_output)

# Launch the app
app.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fc2f8bfad9d3745d6.gradio.live

This share link expires in 72 hours.

In [1]:
# Install required libraries
!pip install gradio pandas numpy scikit-learn

# Import libraries
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Generate synthetic data with added noise to prevent overfitting
def generate_training_data():
    np.random.seed(42)  # For reproducibility
    temperature = np.random.randint(20, 40, 10000)
    wind_speed = np.random.randint(0, 30, 10000)
    noise = np.random.choice([0, 1, -1], size=10000, p=[0.8, 0.1, 0.1])  # Add slight randomness
    crowd_level = [
        "Low" if temp + n < 25 and wind + n > 20 else
        "High" if temp + n > 35 and wind + n < 10 else
        "Medium"
        for temp, wind, n in zip(temperature, wind_speed, noise)
    ]
    data = {
        "Temperature (\u00b0C)": temperature,
        "Wind Speed (km/h)": wind_speed,
        "Crowd Level": crowd_level
    }
    return pd.DataFrame(data)

# Prepare data for training
training_data = generate_training_data()
label_map = {"Low": 0, "Medium": 1, "High": 2}  # Map crowd levels to numeric labels
training_data["Crowd Level"] = training_data["Crowd Level"].map(label_map)

# Split data into features and target
X = training_data[["Temperature (\u00b0C)", "Wind Speed (km/h)"]]
y = training_data["Crowd Level"]

# Add feature engineering to improve accuracy
X["Temp_Wind_Interaction"] = X["Temperature (\u00b0C)"] * X["Wind Speed (km/h)"]
X["Temp_Squared"] = X["Temperature (\u00b0C)"] ** 2
X["Wind_Squared"] = X["Wind Speed (km/h)"] ** 2

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an optimized Random Forest Classifier with careful tuning
model = RandomForestClassifier(n_estimators=800, max_depth=15, min_samples_split=20, min_samples_leaf=10, random_state=42)
model.fit(X_train, y_train)

# Evaluate model accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Define prediction function
def predict_crowd_level(temp, wind):
    interaction_term = temp * wind
    temp_squared = temp ** 2
    wind_squared = wind ** 2
    prediction = model.predict([[temp, wind, interaction_term, temp_squared, wind_squared]])[0]
    reverse_label_map = {0: "Low", 1: "Medium", 2: "High"}
    return reverse_label_map[prediction]

# Create Gradio interface
with gr.Blocks() as app:
    gr.Markdown("## Predict Beach Crowd Levels")

    # User input sliders
    temp_input = gr.Slider(20, 40, step=1, label="Temperature (\u00b0C)")
    wind_input = gr.Slider(0, 30, step=1, label="Wind Speed (km/h)")

    # Prediction output
    predict_button = gr.Button("Predict Crowd Level")
    crowd_output = gr.Textbox(label="Predicted Crowd Level")

    predict_button.click(predict_crowd_level, inputs=[temp_input, wind_input], outputs=crowd_output)

    # Display training data for reference
    gr.Markdown("### Training Data Sample")
    gr.DataFrame(training_data.head())

# Launch the app
app.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Model Accuracy: 0.99
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b460ecf20dd97f241.gradio.live

This share link